In [26]:
import pandas as pd
import sqlite3
import os
import re

class combine():
    
    def old():

        path = os.getcwd()+'\_94-11\\'
        folders = os.listdir(path)
        files = os.listdir(path)
        combined = pd.DataFrame()
        year_list = []
        year = 0

        for folder in reversed(folders):

            if (os.path.isdir(path+folder)) & (folder != 'Old') & (year not in year_list):
                print (folder)
                for file in sorted(os.listdir(path+folder)):

                    only_file = file
                    file = path+folder+'\\'+file

                    try:
                        year = re.search(r'\d{4}',only_file)[0]
                    except:
                        year = 0

                    try:    
                        name = re.search(r'(_\D+?_)',only_file)[0]
                        name = name.replace('_','')
                    except:
                        name = None

                    if (year != 0) & (file[-3:] != 'zip'):

                        if year not in year_list:
                            year_list.append(year)

                        #print (year,name)
                        if file[-4:] == 'xlsx':
                            df = pd.read_excel(file)
                            df['Year'] = re.search(r'\d{4}',file)[0]

                            combined = combined.append(df,sort=False)

                        elif file[-3:] == 'csv':

                            df = pd.read_csv(file)
                            df['Year'] = year

                            combined = combined.append(df,sort=False)

        combined.to_csv('fpicb_ab_00-11_combined.csv',index=False)
        print ('Saved to:',os.getcwd())

    def new():
        '''
        2012-2016 is in a different format. Requires different process.
        '''
        path = os.getcwd()+'\_12-16\\'
        folders = os.listdir(path)
        dfc = {}
        count = 0

        for folder in folders:
            '''
            if True:#count % 2 < 1:

                for item in dfc:
                    dfc[item].to_csv(item+str(count)+'.csv',index=False)

                dfc = {}
            '''     
            if os.path.isdir(path+folder):
                print (folder)

                for file in os.listdir(path+folder):
                    if (file[-3:] == 'csv') & \
                    (file[:5] != 'Naics') & \
                    ('unincorporated' not in file) & \
                    ('profit' not in file):# & ('percentage' not in file):

                        #m = re.search('[_]\d{4}', file)
                        #year = m[0][1:5]
                        #print (year)

                        name = file.split('_')
                        #print (len(name),name)

                        df = pd.read_csv(path+folder+'\\'+file,encoding='latin1')

                        if 'percentage' in file:
                            df['Type '] = 'Pct'
                        else:
                            df['Type '] = 'Dollars'

                        if '20M' in file:
                            df['Size '] = '5M-20M'
                        elif ('30K' in file) or ('30k' in file):
                            df['Size '] = '30K-5M'
                        else:
                            df['Size '] = 'Not Specified'

                        if 'expenses' in file:
                            if 'percentage' in file:
                                df.to_csv(path+'Expenses_pct.csv',mode='a',index=False)
                            else:
                                df.to_csv(path+'Expenses.csv',mode='a',index=False)


                        if 'revenue' in file:
                            df.to_csv(path+'Revenue.csv',mode='a',index=False)


                        if 'ratios' in file:
                            df.to_csv(path+'Ratios.csv',mode='a',index=False)


                        if 'Balance' in file:
                            df.to_csv(path+'Balance Sheet.csv',mode='a',index=False)

            count += 1

        for item in dfc:
            dfc[item].to_csv(item+'.csv',index=False)

        del(dfc)
        
        
def save_to_db(dfp):
    print (dfp['Category'].unique())
    con = sqlite3.connect('finperf.db')
    dfp.to_sql('finperf',con=con,schema=None,if_exists='replace',index=False,chunksize=400000)
    
def transform():
    '''
    Clean up the data files, get them ready to put into a SQL DB
    '''
    path = os.getcwd()+'\_12-16\\'
    files = os.listdir(path)
    dfp = pd.DataFrame()
    fmt = pd.read_csv('category_format.csv').set_index('plain').to_dict()['formatted']
    
    for file in files:
        if (file[-3:] == 'csv') and ('Naics' not in file):
            df = pd.read_csv(path+file,encoding='latin1',header=0)

            col_names_fmt = [i[0:i.find('\r\n')] for i in list(df)]
            df.columns = col_names_fmt

            #

            pivot_dict = {}

            #find strings with quartiles and middle
            for name in col_names_fmt:
                try:
                    #strip to just "___quartile" or "middle ___"
                    pivot_dict[name] = re.search(r'(\b(\w* (quart|Quart)\w*)\b)|(\b(\w* (Middle|middle)\w*)\b)',name)[0]

                except:
                    pass

            df.rename(pivot_dict, axis=1, inplace=True)

            #replace Province numbers with name
            names = pd.read_csv('provinces.csv',names=['Province','name'])              
            
            #
            
            instances = [i for i, x in enumerate(list(df)) if x == 'Quality Indicator']

            if (file == 'Balance Sheet.csv') or (file == 'Expenses.csv') or (file == 'Expenses_pct') or (file == 'Revenue.csv'):
                for i in instances:

                    #grab first/last 3 cols to make the index
                    index_cols = pd.concat([df.iloc[:,:3],df.iloc[:,-3:]], axis=1, sort=False)
                    tmp = pd.concat([index_cols,df.iloc[:,i-1:i+5]], axis=1, sort=False)
                    tmp['Category'] = list(tmp)[6]
                    tmp.drop('Quality Indicator', axis=1, inplace=True)
                    tmp.columns = ['Incorporation Status', 'NAICS', 'Province', 'Year ', 'Type', 'Size', 'Value', 'Bottom Quartile', 'Lower Middle', 'Upper Middle', 'Top Quartile','Category']
                    tmp = tmp[tmp['Incorporation Status'] == 2]
                    tmp['Dataset'] = file[:-4]
                    
                    #Hack so these two pop up together
                    #tmp['Dataset'] = tmp['Dataset'].replace({'Expenses':'Revenue & Expenses','Revenue':'Revenue & Expenses'})
                    
                    tmp.replace(to_replace={99999.9:'-','99999.9':'-'},inplace=True)
                    tmp['Province'].astype(int)
                    tmp = tmp.merge(names,how='left')    
                    tmp.drop('Province',axis=1,inplace=True)
                    tmp.rename({'name':'Province'},axis=1,inplace=True)

                    tmp['Category'] = tmp['Category'].replace(fmt)

                    dfp = dfp.append(tmp)

            if (file == 'Ratios.csv'):
                df.rename({'Revenue to equity ratioRatio recettes-capitaux propre':'Revenue to equity ratio'}, axis=1, inplace=True)
                key_cols = ['Current ratio',
                            'Debt to equity ratio',
                            'Interest coverage ratio',
                            'Debt ratio',
                            'Revenue to equity ratio',
                            'Revenue to closing inventory ratio',
                            'Current debt to equity (%)',
                            'Net profit to equity (%)',
                            'Net fixed assets to equity (%)',
                            'Gross margin (%)',
                            'Return on total assets (%)',
                            'Collection period for accounts receivable (days)'
                           ]

                instances = [i for i, x in enumerate(list(df)) if x in key_cols]

                for i in instances:
                    index_cols = pd.concat([df.iloc[:,:3],df.iloc[:,-3:]], axis=1, sort=False)
                    tmp = pd.concat([index_cols,df.iloc[:,i:i+5]], axis=1, sort=False)
                    tmp['Category'] = list(tmp)[6]
                    tmp.columns = ['Incorporation Status', 'NAICS', 'Province', 'Year ', 'Type', 'Size', 'Value', 'Bottom Quartile', 'Lower Middle', 'Upper Middle', 'Top Quartile','Category']
                    tmp['Dataset'] = file[:-4]
                    tmp.replace(to_replace={99999.9:'-','99999.9':'-'},inplace=True)
                    
                    tmp = tmp.merge(names,how='left')
                    tmp.drop('Province',axis=1,inplace=True)
                    tmp.rename({'name':'Province'},axis=1,inplace=True)
   
                    dfp = dfp.append(tmp)

            regions = list(df)[3]
            
    save_to_db(dfp)

#combine.new()
transform()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2808: DtypeWarning: Columns (118) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2808: DtypeWarning: Columns (21,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


['Total assets' '&nbsp;&nbsp;&nbsp;&nbsp;Total current assets'
 '&nbsp;&nbsp;&nbsp;&nbsp;Accounts receivable'
 '&nbsp;&nbsp;&nbsp;&nbsp;Closing inventory'
 '&nbsp;&nbsp;&nbsp;&nbsp;Other current assets'
 '&nbsp;&nbsp;&nbsp;&nbsp;Net tangible and intagible assets'
 '&nbsp;&nbsp;&nbsp;&nbsp;All other assets and adjustments'
 'Total liabilities' '&nbsp;&nbsp;&nbsp;&nbsp;Total current liabilities'
 '&nbsp;&nbsp;&nbsp;&nbsp;Current bank loans'
 '&nbsp;&nbsp;&nbsp;&nbsp;Other current liabilities'
 '&nbsp;&nbsp;&nbsp;&nbsp;Long term liabilities' 'Total equity'
 'Cost of Sales (direct expenses)'
 '&nbsp;&nbsp;&nbsp;&nbsp;Wages and Benefits'
 '&nbsp;&nbsp;&nbsp;&nbsp;Purchases, materials and sub-contracts'
 'Opening Inventory' 'Closing Inventory' 'Operating expenses (indirect) '
 '&nbsp;&nbsp;&nbsp;&nbsp;Labour and Commissions'
 '&nbsp;&nbsp;&nbsp;&nbsp;Amortization and depletion'
 '&nbsp;&nbsp;&nbsp;&nbsp;Repairs and maintenance'
 '&nbsp;&nbsp;&nbsp;&nbsp;Utilities and telephone/telecommunicat

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:2130: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)


In [18]:
import sqlite3
import pandas as pd
con = sqlite3.connect('finperf.db')
con.execute("VACUUM")
con.close()
df = pd.read_sql_query('select * FROM finperf WHERE NAICS IN (813) AND Province IN ("Alberta") AND Dataset IN ("Balance Sheet") AND "Year " IN (2016) limit 20', con)
df

,Incorporation Status,NAICS,Year,Type,Size,Value,Bottom Quartile,Lower Middle,Upper Middle,Top Quartile,Category,Dataset,Province
0,2,813,2016,Dollars,30K-5M,755.8,115.8,270.6,511.9,2124.9,Total assets,Balance Sheet,Alberta
1,2,813,2016,Dollars,30K-5M,321.4,67.9,148.0,278.6,791.1,&nbsp;&nbsp;&nbsp;&nbsp;Total current assets,Balance Sheet,Alberta
2,2,813,2016,Dollars,30K-5M,23.7,3.9,6.4,13.8,70.8,Accounts receivable,Balance Sheet,Alberta
3,2,813,2016,Dollars,30K-5M,1.7,0.3,0.8,1.2,4.4,Closing inventory,Balance Sheet,Alberta
4,2,813,2016,Dollars,30K-5M,296.0,63.6,140.8,263.6,716.0,Other current assets,Balance Sheet,Alberta
5,2,813,2016,Dollars,30K-5M,236.9,27.3,84.1,137.7,698.7,Net tangible and intagible assets,Balance Sheet,Alberta
6,2,813,2016,Dollars,30K-5M,197.4,20.6,38.5,95.6,635.1,All other assets and adjustments,Balance Sheet,Alberta
7,2,813,2016,Dollars,30K-5M,174.5,22.2,43.7,79.9,552.4,Total liabilities,Balance Sheet,Alberta
8,2,813,2016,Dollars,30K-5M,89.1,15.2,25.9,50.6,264.5,Total current liabilities,Balance Sheet,Alberta
9,2,813,2016,Dollars,30K-5M,4.1,0.5,0.6,2.2,13.0,Current bank loans,Balance Sheet,Alberta
